In [ ]:
# 20250525
# part 2, 

# gened with get_headline_2.2 ,(2.0) & model_3.0
# original prompt in drive

In [1]:
# Cell 1: Imports and Setup
import pandas as pd
from joblib import load
import numpy as np

# Cell 2: Model Loading Function
def load_sector_model(sector_name):
    # Convert sector name to match filename convention
    formatted_name = sector_name.replace(" ", "_")
    model_path = Path(f"sector_models_2/{formatted_name}_model.joblib")
    
    if not model_path.exists():
        raise FileNotFoundError(f"Sector model not found: {model_path}")
        
    return load(model_path)

In [2]:

# Cell 3: Calculation Functions
def calculate_return(row, model):
    coeffs = model['coefficients']
    return (
        coeffs['positive'] * row['Positive'] +
        coeffs['neutral'] * row['Neutral'] +
        coeffs['negative'] * row['Negative'] +
        model['intercept']
    )

def calculate_risk(row):
    return 1 - max(row['Positive'], row['Neutral'], row['Negative'])


In [3]:
# Cell 4: Main Processing Functions
from pathlib import Path

def load_sector_model(sector_name):
    """Load model with proper filename handling"""
    formatted_name = sector_name.replace(" ", "_")
    model_path = Path(f"sector_models_2/{formatted_name}_model.joblib")
    
    if not model_path.exists():
        raise FileNotFoundError(f"Model not found: {model_path}")
    
    return load(model_path)

def calculate_return(row, model):
    """Calculate predicted return using model coefficients"""
    return (
        model['coefficients']['positive'] * row['Positive'] +
        model['coefficients']['neutral'] * row['Neutral'] +
        model['coefficients']['negative'] * row['Negative'] +
        model['intercept']
    )

def calculate_risk(row):
    """Calculate risk as 1 - max sentiment score"""
    return 1 - max(row['Positive'], row['Neutral'], row['Negative'])

def run_model_predictions(ticker, sentiment_file):
    """Main prediction function with fallback logic"""
    try:
        # Load data and extract sector name
        df = pd.read_csv(sentiment_file)
        sector_name = Path(sentiment_file).name.split('_')[0]
        
        # Load appropriate sector model
        model = load_sector_model(sector_name)
        
        # Find relevant data
        company_data = df[df['Ticker'] == ticker]
        sector_data = df[df['Ticker'] == 'SECTOR']
        
        # Data fallback logic
        if not company_data.empty:
            data = company_data.iloc[0]
            source = "Company"
        elif not sector_data.empty:
            data = sector_data.iloc[0]
            source = "SECTOR"
            print(f"- Using sector data as fallback for {ticker}")
        else:
            print(f"- No data available for {ticker} or sector")
            return

        # Calculate metrics
        return_val = calculate_return(data, model)
        risk_val = calculate_risk(data)
        
        # Print formatted results
        print(f"\n{' Prediction Results ':=^40}")
        print(f"Ticker:       {ticker}")
        print(f"Source:       {source}")
        print(f"Return:       {return_val:.6f}")
        print(f"Risk:         {risk_val:.6f}")
        print(f"Sector Model: {sector_name.replace('_', ' ')}")
        print(f"Data File:    {Path(sentiment_file).name}")
        print("=" * 40)
        
    except Exception as e:
        print(f"- Error processing {ticker}: {str(e)}")


In [5]:
# Cell 5: Execution with User Input
if __name__ == "__main__":
    ticker = input("Enter ticker symbol (e.g. AAPL): ").strip().upper()
    csv_file = "Basic Materials_sentiments.csv"
    run_model_predictions(ticker, csv_file)


Enter ticker symbol (e.g. AAPL):  APPL


- Using sector data as fallback for APPL

========== Prediction Results ==========
Ticker:       APPL
Source:       SECTOR
Return:       0.003866
Risk:         0.266911
Sector Model: Basic Materials
Data File:    Basic Materials_sentiments.csv
